This is the same thing as getWhissell but using the emotion dictionary from Warriner at al 2013. They have many more words (almost 14,000) but no function words, and they use lemmas on the assumption that only a few plurals are used more often than singulars and as such should not have very distinct emotional profiles. So processing this dictionary for scores will be a bit different. Their three vectors are Valence, Arousal, and Dominance. Dominance is not related to Whissell's Imagery so that third score will have to be neglected for now (though they discuss imageability on p. 1199. Brysbaert et al 2014 provides concreteness scores, and it is there that they discuss choosing lemmas over other forms. For that list, which they don't say how it might relate to this one, they include plurals that are used as different parts of speech or that are used more frequently than the singular [eyes]). So as a first guess lets lemmatize the text, not run a stop list but look up each word and keep it on the Nonelist for now. Lemmatization has its own pitfalls so we need to keep an eye on that. Valence (EE) mean is 5.06, Activation (AA) is 4.21, and Dominance (DD) is 5.18 (on a scale from 1-9, lowest-highest). I will try using SpaCy instead of NLTK, for speed and so it will scale. 

In [2]:
import spacy
import textacy
import re
#import matplotlib.pyplot as plt
#from statistics import mean

targetDoc = "texts/mdg.txt"     # the text to analyze (relative path)
#targetWord = "birds"               # the word type you want to score
#targetWindow = 6                  # this is tokens before and after target

#nlp = spacy.load('en')
#doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')
document = open(targetDoc).read()
document = re.sub("('\s|\s')"," ", document)           #odd thing for Waves.text, too many single quotes
textacy.preprocess.normalize_whitespace(document)
textacy.preprocess_text(document, lowercase=True, no_punct=False)
#textacy.text_utils.KWIC(document, 'on', window_width=35)
doc = textacy.Doc(document)

quotes = textacy.extract.direct_quotations(doc)
for quote in quotes:
    print(quote)

#for sent in doc.sents:
#    for word in sent:
#        if word.text == targetWord:
#            print(sent)  #prints sentences that have the target word in them
            

#trips = textacy.extract.subject_verb_object_triples(doc) # makes a generator; transitives only
#for item in trips:
#    print(item)

#for sent in doc.sents:
#    start_i = sent[0].i


#    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
 #           token.shape_, token.is_alpha, token.is_stop)

(Rainsford, asked, "What island is it?")
(Rainsford, asked, "The old charts call it Ship-Trap Island,'")
(imagination, said, "Pure imagination,")
(imagination, said, "One superstitious sailor can taint the whole ship's company with his fear.")
(he, thought, "It's so dark,")
(he, thought, "Where there are pistol shots, there are men. Where there are men, there is food,")
(he, said, "It is a very great pleasure and honor to welcome Mr. Sanger Rainsford, the celebrated hunter, to my home.")
(he, said, "So am I.")
(he, said, "Come,")
(he, said, "we shouldn't be chatting here. We can talk later. Now you want clothes, food, rest. You shall have them. This is a most -- restful spot.")
(Follow Ivan, said, "Follow Ivan, if you please, Mr. Rainsford,")
(Follow Ivan, said, "I was about to have my dinner when you came. I'll wait for you. You'll find that my clothes will fit you, I think.")
(he, suggested, "You'll have a cocktail, Mr. Rainsford,")
(Half, said, "We do our best to preserve the amenit

    c = nltk.ConcordanceIndex(doc_text.tokens, key = lambda s: s.lower())
basically indexes the words for every token of the text. The 'offset' function returns the list of token offset positions for a word. (c.\_tokens) is the list of words, so a subscript of that will access the exact word at that position. 
Next, open the DA file and read it into a dictionary. Each line is split so the word is the key and the three Whissell numbers is the value. Lookup is simple: get(key) returns value: daffect.get(c.\_tokens[335]). Then split the value strings into three floating point numbers because math.

In [ ]:
c = nltk.ConcordanceIndex(doc_text.tokens, key = lambda s: s.lower())
locations = c.offsets(targetWord)

with open("texts/DAL.txt") as dictionary_affect_file:
   daffect = dict(line.rstrip().split(",", 1) for line in dictionary_affect_file)

alleevals = []                   #declare lists of vals for every token found in text
allaavals = []
alliivals = []
allNonelist = []                 # keep all nonelist for token report
allAvgeevals = []                # keep list of avg vals for plotting
allAvgaavals = []
allAvgiivals = []

for location in locations:       #iterate over list of concordance hit offsets
    print (c._tokens[location], "offset =", location)
    phrase = []                  #keep a list of tokens in the phrase
    nonelist = []                #new val lists for each interation of a location
    eevals = []
    aavals = []
    iivals = []     
    for i in range(0-targetWindow, 1+targetWindow):        #iterate over range from target offset, range extends up to but no including
        #print (c._tokens[location+i], daffect.get(c._tokens[location+i]))
        #print (c._tokens[location-i], daffect.get(c._tokens[location-i]))
        phrase.append(c._tokens[location+i]) 
         
        if daffect.get(c._tokens[location+i]) is None:
            nonelist.append(c._tokens[location+i])         # prepare report of words not in DAL
        else:
            eeval, aaval, iival = daffect.get(c._tokens[location+i]).split(",")
            eevals.append(float(eeval))                    # have to convert these to fp numbers
            aavals.append(float(aaval))
            iivals.append(float(iival))
            #print (c._tokens[location+i], "eevals = ", eevals)
            #print ("eevals length =",len(eevals))
            
    alleevals.extend(eevals)                 #add vals from this token onto alltokens list
    allaavals.extend(aavals)                 #to calculate mean of alltokens 
    alliivals.extend(iivals)                 #problem: target word gets added each time
    allNonelist.extend(nonelist)
    #print (alleevals) #check to see
    
    avgeevals = round(mean(eevals), 4)       # rounded to 4 decimal places like Whissell
    avgaavals = round(mean(aavals), 4)       # rounded to 4 decimal places like Whissell
    avgiivals = round(mean(iivals), 2)       # rounded to 2 decimal places like Whissell
    
    allAvgeevals.append(avgeevals - 1.85)  # .extend causes a TypeError: 'float' object is not iterable
    allAvgaavals.append(avgaavals - 1.67)  # We subtract the balanced corpus mean to give variance
    allAvgiivals.append(avgiivals - 1.525)
    
    # here follow the token reports: all the words, the No Vals list, and the scores
    print (" ".join(word for word in phrase))
    print ("No Values for", nonelist)
    print ("EE =", avgeevals, "AA =", avgaavals, "II =", avgiivals, "\n") 

hitratio = 100*(round(len(alleevals) / (len(alleevals) + len(allNonelist)), 3))
meanTotaleevals = round(mean(alleevals), 4)
meanTotalaavals = round(mean(allaavals), 4)
meanTotaliivals = round(mean(alliivals), 2)

# here follow the type reports: the target word, its score, and the hit ration. Need anything else?
print ("Type Means for", targetWord) 
print ("EE =", meanTotaleevals)
print ("AA =", meanTotalaavals)
print ("II =", meanTotaliivals)
print ("No values for", list(sorted(set(allNonelist))))
print ("Hit Ratio = %",hitratio)
    


Next step is to display values for each word plotted over the book. Really need to follow her methodology as first guesstimate. "The mean of all values for a sample characterizes the sample as a whole." Whissell 2009, 515. She takes the mean for all word sets involved. For _Much Ado about Nothing_ she takes total speeches from each character (p. 516) and compares values to compare characters. In her other test she examines a police-involved shooting in Ireland, the speech of the shooter and of the later police report. There she focuses on the extreme ends of the emotion scales, and the method is not quite clear. She refers to a 2007 article on "Ruth" that clarifies nothing--she compares to "normative" numbers of strong emotion words in her 1998 corpus but does not provide those numbers anywhere that I found. 
Display: from DAL: on a balanced corpus, mean ee is 1.85, with an sd of .36. So, if we subtract 1.85 from our eevals we will get the distance positive or negative from the mean. Lets try it.

In [ ]:
plt.plot(allAvgeevals, 'g', allAvgaavals, 'r', allAvgiivals, 'orange')
plt.suptitle(targetWord)
plt.show()